In [ ]:
# Feature Engineering para Análise de Violência
# =============================================

# Importações condicionais com fallback
try:
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.feature_selection import SelectKBest, f_classif
    import warnings
    warnings.filterwarnings('ignore')
    print("✅ Todas as bibliotecas importadas com sucesso!")
except ImportError as e:
    print(f"⚠️ Erro de importação: {e}")
    # Fallback para desenvolvimento
    class MockDataFrame:
        def __init__(self, data=None):
            self.data = data or {}
        def groupby(self, col):
            return MockGroupBy()
        def sort_values(self, cols):
            return self
        def fillna(self, value):
            return self
        def to_csv(self, path, index=False):
            print(f"Mock: Salvando em {path}")
    
    class MockGroupBy:
        def __getitem__(self, key):
            return MockSeries()
        def rolling(self, window):
            return MockRolling()
        def pct_change(self):
            return [0.1, 0.2, 0.3]
    
    class MockSeries:
        def first(self):
            return 100000
        def mean(self):
            return 50
    
    class MockRolling:
        def mean(self):
            return [45, 50, 55]
    
    # Mock das bibliotecas
    pd = type('MockPandas', (), {'read_csv': lambda x: MockDataFrame(), 'to_datetime': lambda x: MockDataFrame()})()
    np = type('MockNumpy', (), {
        'sin': lambda x: [0.5, 0.7, 0.9],
        'cos': lambda x: [0.8, 0.6, 0.4],
        'sqrt': lambda x: [1.2, 1.5, 1.8],
        'random': type('MockRandom', (), {
            'uniform': lambda a, b, size: [0.7, 0.8, 0.9]
        })()
    })()
    
    class MockStandardScaler:
        def fit_transform(self, X):
            return X
    
    class MockLabelEncoder:
        def fit_transform(self, x):
            return [0, 1, 2, 3]
    
    class MockSelectKBest:
        def __init__(self, score_func, k):
            self.k = k
        def fit_transform(self, X, y):
            return X
        def get_support(self, indices=True):
            return list(range(min(self.k, 5)))
    
    StandardScaler = MockStandardScaler
    LabelEncoder = MockLabelEncoder
    SelectKBest = MockSelectKBest
    f_classif = lambda x, y: ([1, 2, 3], [0.1, 0.2, 0.3])

# Carregar dados
try:
    df_crimes = pd.read_csv('data/processed/crimes_processed.csv')
    print("✅ Dados carregados com sucesso!")
except:
    print("⚠️ Arquivo não encontrado, usando dados simulados")
    df_crimes = MockDataFrame()

# 1. FEATURES TEMPORAIS
# ====================

# Extrair componentes de data
df_crimes['ano'] = pd.to_datetime(df_crimes['data']).dt.year if hasattr(df_crimes, 'data') else [2024, 2024, 2024]
df_crimes['mes'] = pd.to_datetime(df_crimes['data']).dt.month if hasattr(df_crimes, 'data') else [1, 2, 3]
df_crimes['dia_semana'] = pd.to_datetime(df_crimes['data']).dt.dayofweek if hasattr(df_crimes, 'data') else [0, 1, 2]
df_crimes['dia_mes'] = pd.to_datetime(df_crimes['data']).dt.day if hasattr(df_crimes, 'data') else [1, 15, 28]

# Features cíclicas
df_crimes['mes_sin'] = np.sin(2 * np.pi * df_crimes['mes'] / 12)
df_crimes['mes_cos'] = np.cos(2 * np.pi * df_crimes['mes'] / 12)
df_crimes['dia_semana_sin'] = np.sin(2 * np.pi * df_crimes['dia_semana'] / 7)
df_crimes['dia_semana_cos'] = np.cos(2 * np.pi * df_crimes['dia_semana'] / 7)

print("✅ Features temporais criadas!")


In [ ]:
# Cria features de lag e médias móveis
def create_lag_features(df, group_cols, value_col, lags=[1, 3, 6, 12]):
    """
    Cria features de lag e médias móveis
    """
    df_features = df.copy()
    
    # Ordena por grupo e data
    df_features = df_features.sort_values(group_cols + ['data'])
    
    # Cria lags
    for lag in lags:
        df_features[f'{value_col}_lag_{lag}'] = df_features.groupby(group_cols)[value_col].shift(lag)
    
    # Cria médias móveis
    for window in [3, 6, 12]:
        df_features[f'{value_col}_ma_{window}'] = df_features.groupby(group_cols)[value_col].rolling(window=window).mean().reset_index(0, drop=True)
    
    # Cria diferenças
    df_features[f'{value_col}_diff'] = df_features.groupby(group_cols)[value_col].diff()
    df_features[f'{value_col}_diff2'] = df_features.groupby(group_cols)[value_col].diff(2)
    
    # Cria features de tendência
    df_features[f'{value_col}_trend_3m'] = df_features.groupby(group_cols)[value_col].rolling(window=3).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0]).reset_index(0, drop=True)
    
    return df_features

# Aplica features de lag
df_crimes = create_lag_features(df_crimes, 
                               group_cols=['regiao_administrativa', 'tipo_crime'],
                               value_col='total_ocorrencias')

print("✅ Features de lag e médias móveis criadas")

# Mostra features de lag criadas
lag_features = [col for col in df_crimes.columns if 'lag_' in col or 'ma_' in col or 'diff' in col or 'trend' in col]
print(f"\n📋 Features de lag criadas: {len(lag_features)}")
for feature in lag_features[:10]:  # Mostra primeiras 10
    print(f"  - {feature}")


## 4. FEATURES ESPACIAIS
